# Fichier: LECTURE ET ANALYSE DES CLUSTERS EN STOCK
### Importation des librairies

In [ ]:
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.offline.offline import _plot_html
import plotly.graph_objs as go
print (__version__) # requires version >= 1.9.0
init_notebook_mode(connected=True)

from mpl_toolkits import mplot3d
import matplotlib.pylab as plt
from matplotlib.pylab import rcParams
%matplotlib notebook
#%matplotlib inline

from tslearn.clustering import silhouette_score

# Doc de geo
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
init_notebook_mode(connected=True)

from utils import series_supp as ss
from utils import data_factory as df
from utils import k_mean as km
from utils import k_shape as ks
from utils.statics_func import *
from utils import plot

### Initialise la factory et créé les instances de Support de Series: ss.SeriesSupp

In [ ]:
cwd = os.getcwd()
os.chdir(cwd)
print(cwd)
factory = df.DataFactory(cwd)
global_ploter = plot.Plot(None)

RG24 = ss.SeriesSupp(cwd, factory, "RG24")
RG1 = ss.SeriesSupp(cwd, factory, "RG1")
GW = ss.SeriesSupp(cwd, factory, "GW")

### Importation des datasets depuis les fichiers de stockage csv

In [ ]:
GW.import_dataset()
RG24.import_dataset()

### Création des instances de Clustering
Commenter ou décommenter pour choisir le cluster_GW voulu selon Kshape ou Kmean
* GW: Grand Water -> les piezomètres
* RG: Rain gauge -> les pluviomètres

In [ ]:
#cluster_GW = km.Kmean(GW)
cluster_GW = ks.Kshape(GW)
cluster_RG = km.Kmean(RG24)

## Importation du fichier Pickles contenant le clustering voulu
### Veuillez renseigner le nom du fichier ci dessous

In [ ]:
name_file = openfile_dialog()
cluster_GW.read_cluster(name_file[0])

In [ ]:
#name_file = "Kshape_GW_[2011, 2012, 2013, 2014, 2015][1, 2, 3, 4, 5, 12]_True_5"
#cluster_GW.read_cluster("cluster\\29_06\\" + name_file + ".pkl")

## Affichage des cluster
* Prototypes en rouge
* informations en dessous
* repartition des objets dans les cluster

In [ ]:
cluster_GW.ploter.plot_cluster_light()

In [ ]:
cluster_GW.show_info()
print("rounded values: " + str(cluster_GW.ss.rounded))
cluster_GW.cluster_counter()
cluster_GW.counter
print("tot obj: " + str(len(cluster_GW.ts_clust)))
cluster_GW.capteur_parser()
cluster_GW.style_df("max", cluster_GW.get_captor_distribution_in_cluster())

In [ ]:
cluster_GW.ploter.change_mode(3)
#cluster_GW.cluster_by_fullname

### Silhouette score cluster quality indice
could be huge and long computation due to massive dataset

In [ ]:
#sil = silhouette_score(cluster_GW.ts, cluster_GW.ts_clust, metric="softdtw", metric_params = {"gamma_sdtw": .01})
#print("Silhouette score: " + str(sil))

## Analyse: n_obs est le numero du cluster à observer

### n_obs: numéro du cluster analysé 

In [ ]:
n_obs = 4

In [ ]:
cluster_GW.geo.plotly_3D(cluster_GW.cluster_by_name[n_obs])

In [ ]:
cluster_GW.ploter.plot_histo_2(n_obs)

In [ ]:
cluster_GW.clust_hoverview(n_obs)

In [ ]:
#cluster_GW.cluster_by_fullname

In [ ]:
#cluster_GW.ts_name

### Setup et affiche RG

In [ ]:
#cluster_RG.ss.reset_dataset()
#cluster_RG.ss.years = cluster_GW.ss.years
#cluster_RG.ss.months = cluster_GW.ss.months

In [ ]:
#cluster_RG.ss.split_data_years()
#cluster_RG.ss.split_data_months()
#if cluster_GW.ss.days:
#    cluster_RG.ss.split_data_weeks()

In [ ]:
#cluster_RG.ploter.change_mode(3)
#cluster_RG.ploter.plot_scatter(cluster_RG.ss.tmp_dataset)

In [ ]:
#for k, v in cluster_GW.ss.dataset.items():
#    print(k)

## Setup GW

In [ ]:
print(cluster_GW.ss.tmp_dataset == cluster_GW.ss.dataset)

In [ ]:
cluster_GW.ss.reset_dataset()

In [ ]:
# Smoothing
cluster_GW.ss.dict_smooth()

In [ ]:
# Les trois découpages à commenter ou décommenter selon le besoin
cluster_GW.ss.split_data_years()
#cluster_GW.ss.split_data_months()
cluster_GW.ss.split_data_weeks()

In [ ]:
# Round 0.1
# Normalisation mean = 0 standard deviation = 1
#cluster_GW.ss.dict_norm()

### Analyse un capteur en particulier

In [ ]:
#clts_1 = cluster_GW.get_ts_by_captor("GW_234")
#global_ploter.plot_captor_cluster_cover(clts_1)

In [ ]:
clts = cluster_GW.get_clust_part_for_captor("GW_082")
#cluster_GW.ss.tmp_dataset
#cluster_GW.aff_color()
#global_ploter.plot_scatter_by_capteur_color_cluster(cluster_GW.ss.tmp_dataset, clts)

#### Cluster et RG

In [ ]:
dm = cluster_GW.geo.distance_matrix()
cluster_GW.geo.distance_dict()

In [ ]:
#dm

In [ ]:
RG_captor = cluster_GW.geo.get_minrange_rg(clts[0])
#RG_captor = cluster_GW.geo.get_minrange_rg("GW_126")

In [ ]:
RG_captor

In [ ]:
RG24.reset_dataset()
#RG24.years = cluster_GW.ss.years
RG24.years = list(range(2013, 2016))
RG24.months = cluster_GW.ss.months

RG24.split_data_years()
if cluster_GW.ss.days:
    RG24.split_data_weeks()
else:
    RG24.split_data_months()
    
#RG24.dict_norm() # BUG de Norm

In [ ]:
#RG24.tmp_dataset["24h_RG002_2014_1"]

In [ ]:
RG_captor_values = RG24.get_data_from_captor(str("24h_" + RG_captor))

In [ ]:
global_ploter.change_mode(3)
cluster_GW.aff_color()
global_ploter.plot_scatter_by_capteur_color_cluster(data = cluster_GW.ss.tmp_dataset, dict_clust = clts)
global_ploter.plot_scatter(RG_captor_values)

with open("env\environment.yml") as f:
    res_file = ""
    for line in f:
        res_line = ""
        for letter in line:
            continu = True
            if letter != "=" and continu:
                res_line.append(letter)
            else:
                continu = False
        